# Machine Problem  1 - Path Planning
### **Cruz, Rianna**   
### **Leal, Lois Anne**

**Problem Description**: 
Consider the problem of finding the shortest path between two points on a plane that has polygonal obstacles. This is an idealization of the problem a robot has to solve to navigate it way around a crowded environment.

**Solution Proper**

**1. Setting up the State Space**

expected output: a 100 x 200 grid

In [1]:
grid= [ [0, 0, 0, 0, 0],
        [0, 1, 1, 1, 0],
        [0, 1, 0, 1, 0],
        [0, 1, 1, 1, 0],
        [0, 0, 0, 0, 0],
      ]
print(grid) 

[[0, 0, 0, 0, 0], [0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [0, 1, 1, 1, 0], [0, 0, 0, 0, 0]]


**2. Convert from Grid to Graph Dictionary**

This does the trick. For each cell in the grid, we see them as nodes of the graph. In the code below, what it does is that, for each node in the grid, we consider as a parent node and its children as the possible moves which are left, right, top and bottom. 

***Note***: Cells that belong to the side of the polygon obstacles are not included in the parent node and also in the children of the other nodes.

In [4]:
from collections import defaultdict 

def addEdge(graph,u,v): 
    graph[u].append(v)


def createGraph(grid):
    #get the size of the grid
    rows = len(grid) 
    cols = len(grid[0])

    graph = defaultdict(list) 
    for row in range(rows):
        for col in range(cols):
            if col != 0 and grid[row][col]!=1:
                addEdge(graph,(row,col),(row,col-1)) #left
            if col < cols-1 and grid[row][col]!=1:
                addEdge(graph,(row,col),(row,col+1)) #right
            if row != 0 and grid[row][col]!=1:
                addEdge(graph,(row,col),(row-1,col)) #top
            if row < rows-1 and grid[row][col]!=1:
                addEdge(graph,(row,col),(row+1,col)) #bottom
    
    return graph

graph = createGraph(grid)   

In [5]:
#print graph 
for k,v in graph.items():
    print(k,':',v)

(0, 0) : [(0, 1), (1, 0)]
(0, 1) : [(0, 0), (0, 2), (1, 1)]
(0, 2) : [(0, 1), (0, 3), (1, 2)]
(0, 3) : [(0, 2), (0, 4), (1, 3)]
(0, 4) : [(0, 3), (1, 4)]
(1, 0) : [(1, 1), (0, 0), (2, 0)]
(1, 4) : [(1, 3), (0, 4), (2, 4)]
(2, 0) : [(2, 1), (1, 0), (3, 0)]
(2, 2) : [(2, 1), (2, 3), (1, 2), (3, 2)]
(2, 4) : [(2, 3), (1, 4), (3, 4)]
(3, 0) : [(3, 1), (2, 0), (4, 0)]
(3, 4) : [(3, 3), (2, 4), (4, 4)]
(4, 0) : [(4, 1), (3, 0)]
(4, 1) : [(4, 0), (4, 2), (3, 1)]
(4, 2) : [(4, 1), (4, 3), (3, 2)]
(4, 3) : [(4, 2), (4, 4), (3, 3)]
(4, 4) : [(4, 3), (3, 4)]


**3. Search Algorithms**

**I. Breadth First Search Implementation**  